In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import dendrogram , linkage, cut_tree

In [2]:
df = pd.read_csv('../Datasets/DATA_2.02_HR.csv')
df.head()

,S,LPE,NP,ANH,TIC,Newborn
0,0.38,0.53,2,157,3,0
1,0.80,0.86,5,262,6,0
2,0.11,0.88,7,272,4,0
3,0.72,0.87,5,223,5,0
4,0.37,0.52,2,159,3,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   S        2000 non-null   float64
 1   LPE      2000 non-null   float64
 2   NP       2000 non-null   int64  
 3   ANH      2000 non-null   int64  
 4   TIC      2000 non-null   int64  
 5   Newborn  2000 non-null   int64  
dtypes: float64(2), int64(4)
memory usage: 93.9 KB


# Hierarchical Cluster Analysis

## Capture the Clusters

In [4]:
# the scale function automatically performs data normalization on all your variables
# It substracts the mean and divide by the std of the variable.


# Newborn feature is not relevant to determine who is leaving the company
testdata =df[['S', 'LPE', 'NP', 'ANH', 'TIC']].copy() 

testdata = scale(testdata)
d = pdist(testdata)
# linkage function performs hiearchical clustering, we pass it the distances, 
# and we set the method argument to weighted
hcward = linkage(d, method='ward')
# Finally: we assign our points to our k=4 clusters
df['groups'] = cut_tree(hcward, n_clusters=4)

In [5]:
# We compute the mean of each group
aggdata = df.groupby('groups').mean()
aggdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4 entries, 0 to 3
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   S        4 non-null      float64
 1   LPE      4 non-null      float64
 2   NP       4 non-null      float64
 3   ANH      4 non-null      float64
 4   TIC      4 non-null      float64
 5   Newborn  4 non-null      float64
dtypes: float64(6)
memory usage: 224.0 bytes


In [6]:
# One thing we would like to have is the proportion of our data 
# that is in each cluster
# we create a variable called proptemp which computes the number of observations 
# in each group (using the S variable, but you can take any.)
proptemp = df[['S', 'groups']].groupby('groups').count()

In [7]:
# HR distribution through out groups
aggdata['proportion'] = proptemp.S / sum(proptemp.S)

In [8]:
aggdata.sort_values(by='proportion', ascending=False, inplace=True)
aggdata

,S,LPE,NP,ANH,TIC,Newborn,proportion
groups,,,,,,,
0,0.407345,0.510762,2.003571,143.645238,2.998810,0.053571,0.4200
2,0.103340,0.871818,6.201581,276.300395,4.100791,0.047431,0.2530
1,0.818262,0.923088,4.603272,247.498978,5.204499,0.051125,0.2445
3,0.517091,0.730061,4.133333,208.066667,3.739394,0.066667,0.0825


## Clusters Profiling

So let's make our aggdata more human friendly

In [9]:
columns = {'S': 'Satisfaction',
          'LPE': 'Evaluation',
          'NP': '#Projects',
          'ANH': 'Utiliz',
          'TIC': 'Time'}
aggdata.rename(columns=columns, inplace=True)
aggdata

,Satisfaction,Evaluation,#Projects,Utiliz,Time,Newborn,proportion
groups,,,,,,,
0,0.407345,0.510762,2.003571,143.645238,2.998810,0.053571,0.4200
2,0.103340,0.871818,6.201581,276.300395,4.100791,0.047431,0.2530
1,0.818262,0.923088,4.603272,247.498978,5.204499,0.051125,0.2445
3,0.517091,0.730061,4.133333,208.066667,3.739394,0.066667,0.0825


First let's decide what variables to include or exclude in our clusters profiling:

1. Newborn should definitely be excluded from the profiling since we cannot act on it
2. We should note that the level of satisfaction is a consequence of everything else. We also cannot act on it directly. So it is a consequence and not a driver of managerial impact.
3. The rest of the variables are actionable in practice and should be included in our profiling

In [11]:
# Let's move then newborn column to the end
cols = aggdata.columns.tolist()
cols = cols[:-2] + [cols[-1]] + [cols[-2]]
aggdata = aggdata.reindex(columns=cols)
aggdata

,Satisfaction,Evaluation,#Projects,Utiliz,Time,proportion,Newborn
groups,,,,,,,
0,0.407345,0.510762,2.003571,143.645238,2.998810,0.4200,0.053571
2,0.103340,0.871818,6.201581,276.300395,4.100791,0.2530,0.047431
1,0.818262,0.923088,4.603272,247.498978,5.204499,0.2445,0.051125
3,0.517091,0.730061,4.133333,208.066667,3.739394,0.0825,0.066667


Now let's perform some profiling:

1. The first segment or group 0 didn't do many projects on average and was underutilized. It is also a segment where employees have been in the company for a shorter time than average. Let's call them: **"Low Performer"**
2. The second segment has a good utilization rate, and has been in the company for long time. They are those who work the most, but also they are less satisfied. The two events could be related so let's call those employees **"The Burned"** ones
3. The third segment is very similar to the second segment, with good evaluations and high utilization. But in contrast, they are very satisfied. Let's call them **"the High Potential"**
4. And finally, the last segment doesn't have any distinctive characteristics. Let's call them the **"Misc"** segment.

In [12]:
aggdata['cluster'] = ['Low Perf.', 'Burned Out', 'High Potential', 'Misc']
aggdata

,Satisfaction,Evaluation,#Projects,Utiliz,Time,proportion,Newborn,cluster
groups,,,,,,,,
0,0.407345,0.510762,2.003571,143.645238,2.998810,0.4200,0.053571,Low Perf.
2,0.103340,0.871818,6.201581,276.300395,4.100791,0.2530,0.047431,Burned Out
1,0.818262,0.923088,4.603272,247.498978,5.204499,0.2445,0.051125,High Potential
3,0.517091,0.730061,4.133333,208.066667,3.739394,0.0825,0.066667,Misc


From the above table we can already derive some actionable conclusions :
* 24,5% of our high potential employees, who also have the highest satisfaction score are leaving the company. It could be that one of our competitors poaches our best employees by offering better salaries or growth opportunities or bc the employees didn't plan to stay longer (both are exogenious reasons) or bc of the salary growth?. We should definetely as a manager investigate more this matter and take the necessary actions to reduce the fluctuation of this cluster.

In [14]:
# Let's see how the actionable variables correlate with HR fluctuation
aggdata.corr().loc['proportion'][0:-3]

Satisfaction   -0.176528
Evaluation     -0.497023
#Projects      -0.500669
Utiliz         -0.461711
Name: proportion, dtype: float64

As we can see there is a considerable positive correlation between time spend in the company and statisfaction. 